# <center>Atelier Scraping </center>

<div style="text-align: center;">
    <img src="./images/scrappeur.png" width="600" height="300">
</div>

## Introduction

Commençons par mettre en place les bases du scraping. 

Scraper, c'est savoir lire ce qui se cache derrière un site. Avec un simple clic droit et `inspecter`, on arrive assez simplement à accéder au code HTML de la page.

<div style="text-align: center;">
    <img src="./images/du_site_au_html.png" width="800" height="400">
</div>

Le scraping consiste donc à analyser le code source d'une page pour diverses applications :  

- Localiser des éléments et intérargir avec pour l'automatisation de tâches répétitives (comme des boutons par exemple)
- Extraire différentes informations (ce que nous allons voir dans cet atelier)

____

Ensuite, il faut choisir un outil de scraping : ***Selenium*** ou ***BeautifulSoup*** avec lequel on va pouvoir récupérer les différents éléments d'une page web.

- Selenium est utile pour les pages web dynamiques où le contenu est généré via JavaScript, nécessitant des interactions utilisateur telles que les clics, le défilement, ou la saisie de texte.

- BeautifulSoup est une bibliothèque Python utilisée principalement pour parser (analyser) des documents HTML et XML. Elle est utile pour extraire des données structurées à partir de pages web statiques.

De manière générale, on préfèrera utiliser Selenium qui permet plus d'actions, mais beautifulsoup reste une option importante. Dans, cette activité on présente donc les deux modules pour se faire la main.

___

## Activité 1a : Récupérer une énigme du Professeur Layton avec Selenium

Nous allons voir comment nous pouvons simplement récupérer les éléments principaux d'une page wiki pour s'en faire une base de données. Pour cela, nous allons nous connecter sur https://professeur-layton.fandom.com/fr/wiki/En_queue_de_poisson. 

L'idée dans un premier temps sera de récupérer :
- le titre
- le numéro de l'énigme
- l'énoncé
- la solution
- la résolution

#### Import des modules python

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd

#### Lancer l'ouverture du navigateur et la connexion à l'énigme

In [2]:
# Création d'une instance des options Chrome
chrome_options = Options()

# Définition des options Chrome 
chrome_options.add_argument('--disable-search-engine-choice-screen') 
chrome_options.add_argument('--disable-infobars')
# Création d'une nouvelle instance du navigateur Chrome avec les options spécifiées
driver = webdriver.Chrome(options=chrome_options)

# On renseigne l'URL que l'on veut scraper et on l'injecte dans le driver :
url = "https://professeur-layton.fandom.com/fr/wiki/En_queue_de_poisson"
driver.get(url)

____

Si tout a bien fonctionné, vous voyez normalement une fenêtre chrome s'ouvrir à l'URL que nous avons fourni. 

Le site utilise des cookies, nous pourrions simplement cliquer sur "TOUT ACCEPTER" ou "TOUT REFUSER" mais nous allons utiliser selenium pour réaliser une de ces actions. Les deux prochaines cellules de codes sont très spécifiques au site que nous essayons de scraper, nous n'allons pas nous attarder spécialement dessus : 

In [3]:
cookie = driver.find_element(By.CLASS_NAME ,'NN0_TB_DIsNmMHgJWgT7U')
cookie.click()

Ensuite, nous allons scroller légèrement pour que le programme puisse localiser les différents éléments du site (étant donné la vidéo de pub assez grande en haut de la page web) :

In [4]:
# Récupérer la hauteur totale de la page
total_height = driver.execute_script("return document.body.scrollHeight")

# Calculer la hauteur de défilement (20% dans cet exemple)
scroll_height = total_height * 0.20

# Scroller de 10%
driver.execute_script(f"window.scrollBy(0, {scroll_height});")

___

Nous allons maintenant voir comment récupérer les informations de la consigne : le titre, le numéro de l'énigme, l'énoncé, la solution et la résolution.

#### Récupération astucieuse des éléments Web

Voici un petit [Formulaire HTML](formulaire_html.md). Il permet rapidemment d'apprendre ou se remémorer les balises principales permettant de lire du HTML et d'ainsi repérer les différents éléments d'une page web.


C'est ici que l'on introduit les différents sélecteurs d'éléments Web : 

- ID = "id"
- NAME = "name"
- XPATH = "xpath"
- LINK_TEXT = "link text"
- PARTIAL_LINK_TEXT = "partial link text"
- TAG_NAME = "tag name"
- CLASS_NAME = "class name"
- CSS_SELECTOR = "css selector"

In [5]:
# Récupération du titre :
title = driver.find_element(By.ID, 'firstHeading')

# Récupération du numéro de l'énigme :
num_enigme = driver.find_element(By.XPATH ,'//*[@id="mw-content-text"]/div/div[1]/div[2]/table/tbody/tr[4]/td')

# Récupération de l'énoncé de l'énigme :
enonce = driver.find_element(By.ID, "Énoncé")
enigme_enonce = enonce.find_elements(By.XPATH, "//span[@class='mw-headline' and @id='Énoncé']/ancestor::h2/following-sibling::p[following-sibling::h2]")

# Récupération de la réponse :
reponse = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[8]')

resolution = driver.find_element(By.ID, "Résolution")
resolution_enonce = resolution.find_elements(By.XPATH, "//span[@class='mw-headline' and @id='Résolution']/ancestor::h3/following-sibling::p[following-sibling::h3]")

Observez ce que contiennent nos variables ainsi que leur type : 

In [6]:
title

<selenium.webdriver.remote.webelement.WebElement (session="b4bd56e47bb420f6567fc8602fb8310f", element="f.E9E21ADD1B4BBA7320E52750FAA7022C.d.409A9EA6D88A677D28A825312917BE31.e.98")>

In [7]:
type(title)

selenium.webdriver.remote.webelement.WebElement

In [8]:
resolution_enonce

[<selenium.webdriver.remote.webelement.WebElement (session="b4bd56e47bb420f6567fc8602fb8310f", element="f.E9E21ADD1B4BBA7320E52750FAA7022C.d.409A9EA6D88A677D28A825312917BE31.e.107")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b4bd56e47bb420f6567fc8602fb8310f", element="f.E9E21ADD1B4BBA7320E52750FAA7022C.d.409A9EA6D88A677D28A825312917BE31.e.108")>]

In [9]:
type(resolution_enonce)

list

#### Lecture des éléments Web

Vous avez probablement remarqué que nous avons utilisé `find_element` dans certains cas et `find_elements` dans d'autres. 

Le choix entre ces deux méthodes dépend de ce que l'on cherche à obtenir. 

Si nous voulons extraire un seul élément, comme un titre, nous utiliserons `find_element`, car nous ne souhaitons récupérer qu'un seul élément. 

En revanche, lorsque nous cherchons à obtenir plusieurs éléments, comme les balises de texte pour un énoncé ou la résolution d'une énigme, nous utiliserons `find_elements`, car cela permet de récupérer plusieurs éléments à la fois.

Cette façon de récupérer les éléments va avoir un impact sur la manière de les lire. La méthode `.text` permet d'obtenir le texte d'un WebElement.
Cependant, si vous avez une liste de WebElement, la méthode `.text` ne sera pas directement accessible, d'où le code suivant : 


In [10]:
# Extraction du texte du WebElement title
title = title.text

# Extraction du texte du WebElement num_enigme
num_enigme = num_enigme.text

# Extraction du texte de la liste de WebElements enigme_enonce
enigme_enonce = [elem.text for elem in enigme_enonce]
enigme_enonce = "".join(enigme_enonce)

# Extraction du texte du WebElement reponse
reponse = reponse.text

# Extraction du texte de la liste de WebElements resolution_enonce
resolution_enonce = [elem.text for elem in resolution_enonce]
resolution_enonce = "".join(resolution_enonce)

#### Stockage dans un DataFrame

In [11]:
# Initialiser le dictionnaire de listes
data = {
    'title': title,
    'number': num_enigme,
    'enonce': enigme_enonce,
    'solution': reponse,
    'resolution': resolution_enonce
}
# Afficher le dictionnaire
print(data)

{'title': 'En queue de poisson', 'number': '053', 'enonce': 'Alors que vous aviez le dos tourné, quelqu\'un a englouti le poisson que vous vous étiez préparé pour le dîner. Trois frères se trouvent à proximité des lieux du crime. Voici ce qu\'ils ont a dire :A : "Oui je l\'ai mangé. C\'était drôlement bon !"\nB : "J\'ai vu A manger le poisson !"\nC : "B et moi n\'y avons pas touché."L\'un d\'entre eux vous ment, mais qui ?', 'solution': 'La réponse est C.', 'resolution': "Le menteur est le frère C. A et C se sont partagés votre dîner.La réponse devient évidente quand on réalise que si A ment, alors B ment obligatoirement. Le même raisonnement a lieu si l'on considère que B ment. La seule réponse possible est donc que C est en train de mentir, ce qui implique que C a également touché au poisson."}


____

<div style="text-align: center;">
    <img src="./images/learner_scraping.png" width="300" height="300">
</div>

Et voilà c'est fini ! On a réalisé notre premier scraping ! 

___

## Activité 1b : Récupérer une énigme du Professeur Layton avec BeautifulSoup

On va réaliser le même scraping mais cette fois-ci avec BeautifulSoup.

#### Import des modules python

In [12]:
# Import des bibliothèques
from bs4 import BeautifulSoup, Tag
from typing import List
import requests
import random
import pandas as pd

#### Lancement d'une requête permettant de récupérer tout le code source de l'url à scraper

Pour se connecter au site internet, on va agir différement. Cette fois si on va utiliser le module requests qui envoie des requêtes HTTP, ce qui permet de récupérer le contenu des pages web.

In [13]:
url = "https://professeur-layton.fandom.com/fr/wiki/En_queue_de_poisson"
# Envoie d'une requête HTTP GET 
data  = requests.get(url)

La ligne suivante permet de vérifier si la requête a réussi. Le code de statut 200 indique un succès, tandis que d'autres codes (comme 404 ou 500) indiquent des erreurs :

In [14]:
data.status_code

200

On a bien le code 200, on peut donc passer à la récupération du contenu HTML de la page web : 

In [15]:
# Permet de récupérer le contenu de la réponse à la requête GET sous la forme de texte brut (HTML)
data  = requests.get(url).text
# Création d'un objet BeautifulSoup à l'aide du parser html5lib qui va interpréter le HTML brut contenu dans data
soup = BeautifulSoup(data,"html5lib")
#soup = BeautifulSoup(data,"html.parser")

`soup` est l'objet BeautifulSoup résultant qui permet de naviguer et de manipuler facilement la structure du document HTML.

#### Afficher le code source de la page

In [16]:
#print(soup.prettify())

#### Récupération des éléments Web à partir du soup.prettify

In [17]:
# Extraction du titre et de l'URL de l'énigme
title = soup.find('meta', attrs={'property': "og:title"}).get("content")
url_enigme = soup.find('meta', attrs={'property': "og:url"}).get("content")

#print(title, "\n")
#print(url_enigme)

# Extraction du numéro dans la table associée à "Professeur Layton et l'Étrange Village"
numero = soup.find('a', title="Professeur Layton et l'Étrange Village").find_parent('tr').find_next_sibling('tr').find('td').text.strip()
#print(numero, "\n")

# Fonction pour extraire le texte entre un titre donné et le titre suivant
def extract_text_between(start_id: str, start_tag: str, stop_tag: str) -> str:
    # Trouver l'élément de départ à partir de l'ID du span et de son tag parent
    start_element:Tag = soup.find('span', id=start_id).find_parent(start_tag)
    # Initialiser une liste vide pour collecter le texte
    text_list: List[str] = []
    # Itérer sur tous les éléments suivants du même niveau
    for sibling in start_element.find_next_siblings():
        if sibling.name == stop_tag:  # Arrêter si on atteint l'élément de fin
            break
        if sibling.name in ['p', 'ul']:  # Collecter le texte des éléments <p> et <ul>
            text_list.append(sibling.get_text())
    # Joindre le texte collecté en une seule chaîne de caractères
    return "\n".join(text_list)

# Extraction de l'énoncé et de la résolution
enigme_enonce = extract_text_between('Énoncé', 'h2', 'h2')
reponse = extract_text_between('Résolution', 'h3', 'h3')


#print(enigme_enonce)
#print(reponse)

Précisions : 
- Le préfixe "og" dans og:title et og:url fait référence aux Open Graph Protocol (OGP), un standard utilisé pour structurer et enrichir les métadonnées des pages web. Ce protocole a été initialement développé par Facebook, mais il est maintenant largement utilisé par diverses plateformes de réseaux sociaux et moteurs de recherche pour mieux comprendre et afficher les informations d'une page web lorsqu'elle est partagée.

`numero = soup.find('a', title="Professeur Layton et l'Étrange Village").find_parent('tr').find_next_sibling('tr').find('td').text.strip()`:
- Trouve la balise `<a>` où l'attribut title est égal à "Professeur Layton et l'Étrange Village". Cette balise correspond à un lien hypertexte pointant vers "Professeur Layton et l'Étrange Village".
- `.find_parent('tr')` : Trouve l'élément parent de cette balise `<a>` qui est une balise `<tr>` (ligne de tableau).
- `.find_next_sibling('tr')` : Trouve l'élément suivant au même niveau que cette balise `<tr>`, qui est la prochaine balise `<tr>`. Cela correspond à la ligne suivante dans le tableau.
- `.find('td')` : Trouve la première cellule `<td>` de cette nouvelle ligne `<tr>`, qui contient probablement le numéro de l'énigme.
- `.text.strip()` : Extrait le texte contenu dans cette cellule `<td>` et supprime les espaces blancs en début et en fin de texte avec `strip()`.

In [18]:
# Initialiser le dictionnaire de listes
data = {
    'title': title,
    'number': numero,
    'description': enigme_enonce,
    'solution': reponse
}
# Afficher le dictionnaire
print(data)

{'title': 'En queue de poisson', 'number': '053', 'description': 'Alors que vous aviez le dos tourné, quelqu\'un a englouti le poisson que vous vous étiez préparé pour le dîner. Trois frères se trouvent à proximité des lieux du crime. Voici ce qu\'ils ont a dire\xa0:\n\nA\xa0: "Oui je l\'ai mangé. C\'était drôlement bon\xa0!"\nB\xa0: "J\'ai vu A manger le poisson\xa0!"\nC\xa0: "B et moi n\'y avons pas touché."\n\nL\'un d\'entre eux vous ment, mais qui\xa0?\n', 'solution': "Le menteur est le frère C. A et C se sont partagés votre dîner.\n\nLa réponse devient évidente quand on réalise que si A ment, alors B ment obligatoirement. Le même raisonnement a lieu si l'on considère que B ment. La seule réponse possible est donc que C est en train de mentir, ce qui implique que C a également touché au poisson.\n"}


___
On récupère bien le même résultat, c'est réussi !

___

## Activité 2 : A vous de jouer !

Pour cette activité, nous allons créer un tableau plus complet et pour plusieurs énigmes.

Cette fois-ci, on veut un tableau qui contiendra les colonnes suivantes : `title`, `num_enigme`, `url`, `image`, `enigme`, `solution` pour 3 énigmes.

Nous allons fournir la première partie pour sélectionner 3 énigmes. On commence par aller sur une page qui contient les liens vers toutes les énigmes du jeu : 

In [19]:
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"

On se connecte en mode "headless" pour gagner quelques lignes de code :

In [20]:
# Création d'une instance des options Chrome
chrome_options = Options()

# Définition des options Chrome 
chrome_options.add_argument('--disable-search-engine-choice-screen') 
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--headless=new')
# Création d'une nouvelle instance du navigateur Chrome avec les options spécifiées
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

On récupère tous les liens de type href vers les énigmes grâce à un sélecteur type CSS_Selector : 

In [27]:
# Récupérer tous les éléments <a> avec la classe "category-page__member-link"
elements = driver.find_elements(By.CSS_SELECTOR, "a.category-page__member-link")
# Extraire les hrefs des éléments trouvés
hrefs = [element.get_attribute("href") for element in elements]

On extrait aléatoirement une liste de 3 liens (pour éviter d'avoir un code trop long et de surcharger le wiki en requêtes) :

In [28]:
# Sélectionner aléatoirement 5 hrefs
hrefs = random.sample(hrefs, 3)

In [29]:
hrefs

['https://professeur-layton.fandom.com/fr/wiki/Symbole_du_temps',
 'https://professeur-layton.fandom.com/fr/wiki/Pi%C3%A8ces_jaunes',
 'https://professeur-layton.fandom.com/fr/wiki/Dans_le_Petri']

On va ensuite créer une fonction qui pour chaque lien récupère les données qui nous intéressent :